In [43]:
import os
import pandas as pd

In [44]:
# Creating a Pandas DataFrame our our entire Data.
path = './Cralwer Code/Raw_Dataset/'
folders = os.listdir(path)

dataset = {'id': [],
      'title': [],
      'article': []}
i = 1
for folder_name in folders:
    folder = os.listdir(os.path.join(path,folder_name))
    for file_name in folder:
      dataset['id'].append(i+1)
      i += 1
      dataset['title'].append(file_name)
      article = open(os.path.join(path,folder_name,file_name),"r",encoding = 'utf-8')
      dataset['article'].append(article.read())

dataset = pd.DataFrame(dataset)

#### <i> Using Minimum Hashing</i>

In [46]:
from datasketch import MinHash

In [47]:
df = dataset
df['hash'] = "None"
for i,article in enumerate(df['article']):
    m = MinHash()
    for token in article.split():
        m.update(token.encode('utf-8'))
    df.iloc[i,-1] = m

In [ ]:
print(f"Length before Deduplication: {len(df)}")

threshold = 0.8

index_list =[]

for i in range(len(df)):
    hash_val = df.iloc[i,-1]

    for j in range(len(df)):
        if i != j:
            similarity = hash_val.jaccard(df.iloc[j,-1])
            if similarity > threshold:
                index_list.append(df.iloc[j,0])

df = df[~df['id'].isin(index_list)]
print(f"Length after Deduplication: {len(df)}")

In [49]:
for i in range(len(df)):
    file = open(f"./Raw_Dataset_1/temp/File{i+1}.txt","w",encoding="utf-8" )
    file.write(df.iloc[i,2])
    file.close()

#### <i>Using Similarity Hashing </i>

In [50]:
from simhash import Simhash

In [51]:
df = dataset 
df['hash'] = None

# Calculating Sim Hah for 128 bits.
for i,article in enumerate(df['article']):
    df.iloc[i,-1] = Simhash(article, f = 128)

In [ ]:
# Calculating Hamming Distance for each value.
print(f"Length before Deduplication: {len(df)}")

threshold = 80

index_list =[]

for i in range(len(df)):
    hash_val = df.iloc[i,-1]

    for j in range(len(df)):
        if i != j:
            similarity = hash_val.distance(df.iloc[j,-1])
            if similarity > threshold:
                index_list.append(df.iloc[j,0])

df = df[~df['id'].isin(index_list)]
print(f"Length after Deduplication: {len(df)}")

In [53]:
for i in range(len(df)):
    file = open(f"./Raw_Dataset_1/temp/File{i+1}.txt","w",encoding="utf-8" )
    file.write(df.iloc[i,2])
    file.close()

### Using TF-IDF Cosine Similarity

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = dataset
text = df['article']
vectorizer = TfidfVectorizer().fit_transform(text)
vectors = vectorizer.toarray()
print(vectors.shape)

In [56]:
index_list = []
threshold = 0.8
for i,vector1 in enumerate(vectors):
    for j,vector2 in enumerate(vectors):
        if i != j:
            similarity = cosine_similarity([vector1], [vector2])
            if similarity > threshold:
                index_list.append(j)

In [ ]:
print(f"Length before Deduplication:{len(df)}")
df = df[~df['id'].isin(index_list)]
print(f"Length after Deduplication: {len(df)}")

In [58]:
for i in range(len(df)):
    file = open(f"./Raw_Dataset_1/temp/File{i+1}.txt","w",encoding="utf-8" )
    file.write(df.iloc[i,2])
    file.close()

### Fuzzy 

In [59]:
# from fuzzywuzzy import fuzz
from rapidfuzz import fuzz

In [ ]:
df = dataset

text = df['article']
index_list =[]
threshold = 80

for doc1 in range(len(text)):
    for doc2 in range(len(text)):
        if doc1 != doc2:
            similarity = fuzz.ratio(text[doc1],text[doc2])

            if similarity > threshold:
                index_list.append(doc2)

print(len(index_list))

In [ ]:
print(f"Length before Deduplication:{len(df)}")
df = df[~df['id'].isin(index_list)]
print(f"Length after Deduplication:{len(df)}")

In [62]:
for i in range(len(df)):
    file = open(f"./Raw_Dataset_1/temp/File{i+1}.txt","w",encoding="utf-8" )
    file.write(df.iloc[i,2])
    file.close()